# 成人收入数据集 BT-TWD 可行性实验

本 notebook 按步骤运行：加载配置 → 读取数据 → 预处理 → 桶树划分 → 基线与 BTTWD k 折实验 → 桶级分析。

In [1]:
# 步骤0：环境与路径设置
import os, sys
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

# 将项目根目录加入路径，便于导入 bttwdlib
root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
if root_path not in sys.path:
    sys.path.append(root_path)

from bttwdlib import (
    load_yaml_cfg,
    show_cfg,
    load_adult_raw,
    prepare_features_and_labels,
    BucketTree,
    run_kfold_experiments,
    log_info,
    set_global_seed,
)

cfg_path = os.path.join(root_path, 'configs', 'adult_bttwd.yaml')
cfg = load_yaml_cfg(cfg_path)
set_global_seed(cfg.get('SEED', {}).get('global_seed', 42))
log_info('【步骤0摘要】环境准备完毕，路径与随机种子已设置。')

【INFO】【2025-11-22 20:02:42】【配置加载】已读取 e:\yan\组\三支决策\机器学习\BT_TWD\configs\adult_bttwd.yaml
【INFO】【2025-11-22 20:02:44】【步骤0摘要】环境准备完毕，路径与随机种子已设置。


In [2]:
# 步骤1：加载配置
show_cfg(cfg)
log_info('【步骤1摘要】配置文件加载完成，关键参数检查通过。')

【INFO】【2025-11-22 20:02:44】【配置-数据】数据集=adult, k折=5, 目标列=income, 正类=">50K"
【INFO】【2025-11-22 20:02:44】【配置-BTTWD】阈值模式=bucket_wise, 全局模型=xgb, 桶内模型=knn, 后验估计器(兼容字段)=logreg
【INFO】【2025-11-22 20:02:44】【配置-基线】LogReg启用=True, RandomForest启用=True, KNN启用=True, XGBoost启用=True
【INFO】【2025-11-22 20:02:44】【步骤1摘要】配置文件加载完成，关键参数检查通过。


In [3]:
# 步骤2：加载原始数据
df_raw = load_adult_raw(cfg)
display(df_raw.head())
target_col = cfg['DATA']['target_col']
class_counts = df_raw[target_col].value_counts(normalize=True)
ax = class_counts.plot(kind='bar', title='正负类比例')
plt.ylabel('比例')
fig_path = os.path.join(root_path, cfg['OUTPUT']['figs_dir'], 'class_distribution.png')
os.makedirs(os.path.dirname(fig_path), exist_ok=True)
plt.savefig(fig_path, bbox_inches='tight')
plt.close()
log_info('【步骤2摘要】Adult 原始数据加载与基本统计完成。')

【INFO】【2025-11-22 20:02:44】【数据加载完毕】样本数=32561，特征数=14，正类比例=0.24


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


【INFO】【2025-11-22 20:02:44】【步骤2摘要】Adult 原始数据加载与基本统计完成。


In [4]:
# 步骤3：预处理与特征工程
X, y, meta = prepare_features_and_labels(df_raw, cfg)
log_info(f'【预处理】编码特征维度={X.shape[1]}，样本数={X.shape[0]}')
log_info(f"【步骤3摘要】特征预处理完成：连续={len(meta['continuous_cols'])}，类别={len(meta['categorical_cols'])}，编码维度={X.shape[1]}。")

【INFO】【2025-11-22 20:02:44】【预处理】连续特征=6个，类别特征=8个
【INFO】【2025-11-22 20:02:45】【预处理】编码后维度=100
【INFO】【2025-11-22 20:02:45】【预处理】编码特征维度=100，样本数=32561
【INFO】【2025-11-22 20:02:45】【步骤3摘要】特征预处理完成：连续=6，类别=8，编码维度=100。


In [5]:
# 步骤4：构建桶树并检查划分
bucket_tree = BucketTree(cfg['BTTWD']['bucket_levels'], feature_names=df_raw.drop(columns=[cfg['DATA']['target_col']]).columns.tolist())
bucket_ids_full = bucket_tree.assign_buckets(df_raw.drop(columns=[cfg['DATA']['target_col']]))
bucket_df = bucket_ids_full.value_counts().reset_index()
bucket_df.columns = ['bucket_id', 'count']
bucket_df['pos_rate'] = df_raw.groupby(bucket_ids_full)[cfg['DATA']['target_col']].apply(lambda s: (s == cfg['DATA']['positive_label']).mean()).values
display(bucket_df.head())
bucket_df.set_index('bucket_id')['count'].plot(kind='bar', figsize=(12,4), title='桶样本数分布')
fig_bucket = os.path.join(root_path, cfg['OUTPUT']['figs_dir'], 'bucket_metrics_bar.png')
plt.savefig(fig_bucket, bbox_inches='tight')
plt.close()
log_info(f'【步骤4摘要】桶树划分完成，共有 {bucket_ids_full.nunique()} 个叶子桶。')

【INFO】【2025-11-22 20:02:45】【桶树】已为样本生成桶ID，共 16 个组合


,bucket_id,count,pos_rate
0,L1_age=old|L2_education=high,5328,0.235657
1,L1_age=mid|L2_education=high,4218,0.018779
2,L1_age=old|L2_education=mid,4166,0.097167
3,L1_age=mid|L2_education=mid,3530,0.452080
4,L1_age=young|L2_education=high,3216,0.411974


【INFO】【2025-11-22 20:02:45】【步骤4摘要】桶树划分完成，共有 16 个叶子桶。


In [6]:
# 步骤5：运行基线模型 k 折实验
# 基线部分在 run_kfold_experiments 内统一调度
log_info('【步骤5】基线模型将在整体交叉验证中一并运行。')
log_info('【步骤5摘要】基线模型性能将作为后续对比基准。')

【INFO】【2025-11-22 20:02:45】【步骤5】基线模型将在整体交叉验证中一并运行。
【INFO】【2025-11-22 20:02:45】【步骤5摘要】基线模型性能将作为后续对比基准。


In [7]:
# 步骤6：运行 BTTWD k 折实验（含基线）
results = run_kfold_experiments(X, y, df_raw.drop(columns=[cfg['DATA']['target_col']]), cfg)
summary_df = pd.read_csv(os.path.join(root_path, cfg['OUTPUT']['results_dir'], 'metrics_kfold_summary.csv'))
display(summary_df)
summary_df.plot(x='model', kind='bar', figsize=(8,4), title='模型指标对比')
fig_compare = os.path.join(root_path, cfg['OUTPUT']['figs_dir'], 'metrics_compare.png')
plt.savefig(fig_compare, bbox_inches='tight')
plt.close()
log_info('【步骤6摘要】BTTWD 与基线的 k 折结果已生成并保存。')

【INFO】【2025-11-22 20:03:01】【基线-LogReg】整体指标：Precision=0.735, Recall=0.602, F1=0.662, BAC=0.767, AUC=0.907, MCC=0.573, Kappa=0.568
【INFO】【2025-11-22 20:03:28】【基线-RF】整体指标：Precision=0.739, Recall=0.627, F1=0.678, BAC=0.778, AUC=0.906, MCC=0.590, Kappa=0.587
【INFO】【2025-11-22 20:03:34】【基线-KNN】整体指标：Precision=0.708, Recall=0.559, F1=0.625, BAC=0.743, AUC=0.883, MCC=0.529, Kappa=0.523


C:\Users\xzq20.HUANLI\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:158: UserWarning: [20:03:34] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\xzq20.HUANLI\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:158: UserWarning: [20:03:35] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\xzq20.HUANLI\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:158: UserWarning: [20:03:36] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.



【INFO】【2025-11-22 20:03:41】【基线-XGB】整体指标：Precision=0.786, Recall=0.655, F1=0.714, BAC=0.799, AUC=0.929, MCC=0.639, Kappa=0.634
【INFO】【2025-11-22 20:03:41】【K折实验】正在执行第 1/5 折...
【INFO】【2025-11-22 20:03:41】【桶树】已为样本生成桶ID，共 16 个组合


C:\Users\xzq20.HUANLI\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:158: UserWarning: [20:03:41] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


【INFO】【2025-11-22 20:03:42】【BTTWD】全局模型训练完成，用于兜底预测
【INFO】【2025-11-22 20:03:42】【BTTWD】桶 L1_age=mid|L2_education=high：n=3406, pos_rate=0.24, alpha=0.4, beta=0.05
【INFO】【2025-11-22 20:03:42】【BTTWD】桶 L1_age=mid|L2_education=low 样本太少(n=166)，使用全局回退（min_bucket_size=200）
【INFO】【2025-11-22 20:03:42】【BTTWD】桶 L1_age=mid|L2_education=mid：n=2809, pos_rate=0.09, alpha=0.4, beta=0.05
【INFO】【2025-11-22 20:03:42】【BTTWD】桶 L1_age=mid|L2_education=top：n=435, pos_rate=0.44, alpha=0.4, beta=0.05
【INFO】【2025-11-22 20:03:43】【BTTWD】桶 L1_age=old|L2_education=high：n=4272, pos_rate=0.41, alpha=0.45, beta=0.05
【INFO】【2025-11-22 20:03:43】【BTTWD】桶 L1_age=old|L2_education=low：n=270, pos_rate=0.06, alpha=0.4, beta=0.05
【INFO】【2025-11-22 20:03:43】【BTTWD】桶 L1_age=old|L2_education=mid：n=3335, pos_rate=0.21, alpha=0.4, beta=0.05
【INFO】【2025-11-22 20:03:43】【BTTWD】桶 L1_age=old|L2_education=top：n=1086, pos_rate=0.69, alpha=0.55, beta=0.05
【INFO】【2025-11-22 20:03:44】【BTTWD】桶 L1_age=very_old|L2_education=high：n=1870, pos_rate=0

...

【INFO】【2025-11-22 20:03:56】【BTTWD】全局模型训练完成，用于兜底预测
【INFO】【2025-11-22 20:03:56】【BTTWD】桶 L1_age=mid|L2_education=high：n=3361, pos_rate=0.23, alpha=0.4, beta=0.05
【INFO】【2025-11-22 20:03:56】【BTTWD】桶 L1_age=mid|L2_education=low 样本太少(n=173)，使用全局回退（min_bucket_size=200）
【INFO】【2025-11-22 20:03:57】【BTTWD】桶 L1_age=mid|L2_education=mid：n=2870, pos_rate=0.10, alpha=0.4, beta=0.05
【INFO】【2025-11-22 20:03:57】【BTTWD】桶 L1_age=mid|L2_education=top：n=437, pos_rate=0.47, alpha=0.45, beta=0.05
【INFO】【2025-11-22 20:03:57】【BTTWD】桶 L1_age=old|L2_education=high：n=4209, pos_rate=0.42, alpha=0.4, beta=0.05
【INFO】【2025-11-22 20:03:57】【BTTWD】桶 L1_age=old|L2_education=low：n=256, pos_rate=0.06, alpha=0.4, beta=0.05
【INFO】【2025-11-22 20:03:58】【BTTWD】桶 L1_age=old|L2_education=mid：n=3347, pos_rate=0.22, alpha=0.4, beta=0.05
【INFO】【2025-11-22 20:03:58】【BTTWD】桶 L1_age=old|L2_education=top：n=1081, pos_rate=0.69, alpha=0.55, beta=0.05
【INFO】【2025-11-22 20:03:58】【BTTWD】桶 L1_age=very_old|L2_education=high：n=1876, pos_rate=0

C:\Users\xzq20.HUANLI\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:158: UserWarning: [20:04:00] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


【INFO】【2025-11-22 20:04:01】【BTTWD】全局模型训练完成，用于兜底预测
【INFO】【2025-11-22 20:04:01】【BTTWD】桶 L1_age=mid|L2_education=high：n=3400, pos_rate=0.24, alpha=0.4, beta=0.05
【INFO】【2025-11-22 20:04:01】【BTTWD】桶 L1_age=mid|L2_education=low 样本太少(n=165)，使用全局回退（min_bucket_size=200）
【INFO】【2025-11-22 20:04:01】【BTTWD】桶 L1_age=mid|L2_education=mid：n=2777, pos_rate=0.10, alpha=0.4, beta=0.05
【INFO】【2025-11-22 20:04:01】【BTTWD】桶 L1_age=mid|L2_education=top：n=457, pos_rate=0.45, alpha=0.4, beta=0.05
【INFO】【2025-11-22 20:04:02】【BTTWD】桶 L1_age=old|L2_education=high：n=4283, pos_rate=0.40, alpha=0.45, beta=0.05
【INFO】【2025-11-22 20:04:02】【BTTWD】桶 L1_age=old|L2_education=low：n=269, pos_rate=0.05, alpha=0.4, beta=0.05
【INFO】【2025-11-22 20:04:02】【BTTWD】桶 L1_age=old|L2_education=mid：n=3301, pos_rate=0.22, alpha=0.4, beta=0.05
【INFO】【2025-11-22 20:04:02】【BTTWD】桶 L1_age=old|L2_education=top：n=1105, pos_rate=0.68, alpha=0.55, beta=0.05
【INFO】【2025-11-22 20:04:03】【BTTWD】桶 L1_age=very_old|L2_education=high：n=1858, pos_rate=0

,Precision,Recall,F1,BAC,AUC,MCC,Kappa,model
0,0.608084,0.723891,0.660903,0.787957,0.878267,0.544546,0.540711,BTTWD
1,0.735088,0.601964,0.661899,0.766577,0.906751,0.572938,0.568290,LogReg
2,0.738954,0.627088,0.678441,0.778411,0.905658,0.590440,0.587168,RandomForest
3,0.707849,0.558985,0.624670,0.742903,0.883325,0.529310,0.523384,KNN
4,0.785889,0.654891,0.714435,0.799149,0.929218,0.638742,0.634399,XGBoost


【INFO】【2025-11-22 20:04:05】【步骤6摘要】BTTWD 与基线的 k 折结果已生成并保存。


In [8]:
# 步骤7：桶级别分析
bucket_metrics_path = os.path.join(root_path, cfg['OUTPUT']['results_dir'], 'bucket_metrics.csv')
if os.path.exists(bucket_metrics_path):
    bucket_metrics_df = pd.read_csv(bucket_metrics_path)
    display(bucket_metrics_df.head())
    bucket_metrics_df.plot(x='bucket_id', y='pos_rate', kind='bar', figsize=(12,4), title='桶正类比例')
    plt.ylabel('正类比例')
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.savefig(fig_bucket, bbox_inches='tight')
    plt.close()
log_info('【步骤7摘要】桶级指标已整理，可用于局部化分析。')

,bucket_id,n_samples,pos_rate,alpha,beta,train_score,fold
0,L1_age=old|L2_education=high,4272,0.412687,0.45,0.05,0.765723,1
1,L1_age=mid|L2_education=high,3406,0.237522,0.40,0.05,0.655615,1
2,L1_age=old|L2_education=mid,3335,0.213193,0.40,0.05,0.626594,1
3,L1_age=mid|L2_education=mid,2809,0.094340,0.40,0.05,0.393805,1
4,L1_age=young|L2_education=high,2552,0.023903,0.40,0.05,0.271605,1


【INFO】【2025-11-22 20:04:06】【步骤7摘要】桶级指标已整理，可用于局部化分析。


In [9]:
# 步骤8：结果汇总
log_info('【步骤8】检查结果文件与图表。')
print(os.listdir(os.path.join(root_path, cfg['OUTPUT']['results_dir'])))
print(os.listdir(os.path.join(root_path, cfg['OUTPUT']['figs_dir'])))
log_info('【全部步骤完成】Adult 数据集上的 BT-TWD 可行性实验结束。')

【INFO】【2025-11-22 20:04:06】【步骤8】检查结果文件与图表。
['bucket_metrics.csv', 'metrics_kfold_per_fold.csv', 'metrics_kfold_summary.csv', '~$不同参数结果.docx', '不同参数结果.docx']
['bucket_metrics_bar.png', 'class_distribution.png', 'metrics_compare.png']
【INFO】【2025-11-22 20:04:06】【全部步骤完成】Adult 数据集上的 BT-TWD 可行性实验结束。
